In [175]:
# Biblioteca para tratamiento de datos
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración para poder visualizar todas las columnas y filas de los DataFrames
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

## **Fase 1**: limpieza e imputación

In [176]:
# Función para leer csv
def ejecutar_csv(ruta_csv):
    df= pd.read_csv(ruta_csv)
   
    return df

In [177]:
# LLamo la función para leer csv e imprimir las 5 primeras filas 
df_flight = ejecutar_csv('Customer_Flight_Activity.csv')
df_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [178]:
df_loyalty = ejecutar_csv('Customer_Loyalty_History.csv')
df_loyalty.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [179]:
# Imprimir la forma de mi df, numero de filas y columnas 
df_flight.shape

(405624, 10)

In [180]:
df_loyalty.shape

(16737, 16)

In [181]:
# Función para cambiar los nombres de las columnas
def renombrar_columnas_con_guion_bajo(df):
    df.columns = [col.replace(" ", "_") for col in df.columns]
    return df

In [182]:
# llamo la función para uniformizar los nombres de las columnas y combruebo que funciona 
df_loyalty_1 = renombrar_columnas_con_guion_bajo(df_loyalty)
df_loyalty_1.head()

,Loyalty_Number,Country,Province,City,Postal_Code,Gender,Education,Salary,Marital_Status,Loyalty_Card,CLV,Enrollment_Type,Enrollment_Year,Enrollment_Month,Cancellation_Year,Cancellation_Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [183]:
df_loyalty_1.duplicated().sum()

0

In [184]:
# Hago lo mismo para el segundo df
df_flight_1 = renombrar_columnas_con_guion_bajo(df_flight)
df_flight_1.head()

,Loyalty_Number,Year,Month,Flights_Booked,Flights_with_Companions,Total_Flights,Distance,Points_Accumulated,Points_Redeemed,Dollar_Cost_Points_Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [185]:
df_flight_1.duplicated().sum()

1864

In [186]:
# Creo una nueva variable para la unión. 
# Utilizo el merge ya que los dos archivos tienen columnas en común y para preservar todos los datos ya que los dfs tiene cantidad de datos distintos, 
# Unifizo los dos DataFrames poniendo Loyalty como el primero a la izquierda y Flight como el segundo a la derecha
# Utilizo la columna `Loyalty_Number`, que es la columnas que los dos DFs tiene en común para unificar.
df_customers = df_loyalty_1.merge(df_flight_1, left_on='Loyalty_Number', right_on='Loyalty_Number')

df_customers.head()

,Loyalty_Number,Country,Province,City,Postal_Code,Gender,Education,Salary,Marital_Status,Loyalty_Card,CLV,Enrollment_Type,Enrollment_Year,Enrollment_Month,Cancellation_Year,Cancellation_Month,Year,Month,Flights_Booked,Flights_with_Companions,Total_Flights,Distance,Points_Accumulated,Points_Redeemed,Dollar_Cost_Points_Redeemed
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN,2017,1,0,0,0,0,0.0,0,0
1,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN,2017,2,3,0,3,2823,282.0,0,0
2,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN,2017,3,0,0,0,0,0.0,0,0
3,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN,2017,4,0,0,0,0,0.0,0,0
4,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN,2018,10,6,2,8,3352,335.0,465,38


In [187]:
# Miro cuantas la forma del DF unificado 
df_customers.shape

(405624, 25)

In [188]:
# Visualizo si mi DF tiene duplicados y opto por no quitarlos porque veo que en columna mes, existen numeros distintos para cada cliente
df_customers.duplicated().sum()

1864

In [202]:
# Imprimo el DF con 5 columnas aleatórias
df_customers.sample(5)

,Loyalty_Number,Province,City,Postal_Code,Gender,Education,Salary,Marital_Status,Loyalty_Card,CLV,Enrollment_Type,Enrollment_Year,Enrollment_Month,Year,Month,Flights_Booked,Flights_with_Companions,Total_Flights,Distance,Points_Accumulated,Points_Redeemed,Dollar_Cost_Points_Redeemed
81087,444652,Alberta,Calgary,T3E 2V9,Male,College,73479.0,Single,Aurora,12067.46,Standard,2017,8,2018,4,7,0,7,1694,253.50,0,0
404729,464394,British Columbia,Vancouver,V6E 3Z3,Female,Doctor,145343.0,Married,Star,37150.83,Standard,2012,12,2018,6,0,0,0,0,0.00,0,0
328855,445451,British Columbia,Kelowna,V09 2E9,Female,College,73479.0,Single,Star,5308.25,Standard,2012,5,2017,7,11,7,18,3384,338.00,0,0
312613,126097,Ontario,Toronto,P5S 6R4,Male,College,73479.0,Single,Star,4551.06,Standard,2015,4,2018,7,9,5,14,3486,348.00,747,60
192518,509714,British Columbia,Vancouver,V6E 3D9,Male,Bachelor,69317.0,Divorced,Nova,7957.63,Standard,2012,6,2018,2,6,0,6,1032,128.75,0,0


In [190]:
# Miro como están distribuidas las columnas 
df_customers.dtypes

Loyalty_Number                   int64
Country                         object
Province                        object
City                            object
Postal_Code                     object
Gender                          object
Education                       object
Salary                         float64
Marital_Status                  object
Loyalty_Card                    object
CLV                            float64
Enrollment_Type                 object
Enrollment_Year                  int64
Enrollment_Month                 int64
Cancellation_Year              float64
Cancellation_Month             float64
Year                             int64
Month                            int64
Flights_Booked                   int64
Flights_with_Companions          int64
Total_Flights                    int64
Distance                         int64
Points_Accumulated             float64
Points_Redeemed                  int64
Dollar_Cost_Points_Redeemed      int64
dtype: object

In [191]:
# Calculo la media, la mediana y la moda de la Columna 'Salary' para saber que es mejor imputar
media_salary = df_customers['Salary'].mean()
mediana_salary = df_customers['Salary'].median()
moda_salary = df_customers['Salary'].mode()[0]

print(f'La media de la columna Salary es: {np.round(media_salary, 2)}') 
print(f'La mediana de la columna Salary es: {mediana_salary}')
print(f'La moda de la columna Salary es: {np.round(moda_salary, 2)}')

La media de la columna Salary es: 79268.83
La mediana de la columna Salary es: 73479.0
La moda de la columna Salary es: 101933.0


In [192]:
# Imprimo la cantidad de nullos de mi df 
df_customers.isnull().sum()


Loyalty_Number                      0
Country                             0
Province                            0
City                                0
Postal_Code                         0
Gender                              0
Education                           0
Salary                         102672
Marital_Status                      0
Loyalty_Card                        0
CLV                                 0
Enrollment_Type                     0
Enrollment_Year                     0
Enrollment_Month                    0
Cancellation_Year              355560
Cancellation_Month             355560
Year                                0
Month                               0
Flights_Booked                      0
Flights_with_Companions             0
Total_Flights                       0
Distance                            0
Points_Accumulated                  0
Points_Redeemed                     0
Dollar_Cost_Points_Redeemed         0
dtype: int64

In [193]:
# Utilizo la funcion ".abs" para que convierta el valor numerico negativo de la columna en valor absoluto.
df_customers['Salary'] = df_customers['Salary'].abs()

In [194]:
# Imputo la mediana en la columna 'Salary'
media_salary = df_customers['Salary'].median()
df_customers["Salary"] = df_customers["Salary"].fillna(media_salary)

In [195]:
# Elimino columnas imnecesarias con la función .drop --> Country porque son todos de Canada <--
columnas_a_eliminar= ['Country', 'Cancellation_Year','Cancellation_Month' ]
df_customers.drop(columns=columnas_a_eliminar, axis = 1, inplace=True)
print(f"Las columnas {columnas_a_eliminar} han sido eliminadas")

Las columnas ['Country', 'Cancellation_Year', 'Cancellation_Month'] han sido eliminadas


In [196]:
# Compruebo si hay algun valor nulo en mi DF
df_customers.isnull().sum()

Loyalty_Number                 0
Province                       0
City                           0
Postal_Code                    0
Gender                         0
Education                      0
Salary                         0
Marital_Status                 0
Loyalty_Card                   0
CLV                            0
Enrollment_Type                0
Enrollment_Year                0
Enrollment_Month               0
Year                           0
Month                          0
Flights_Booked                 0
Flights_with_Companions        0
Total_Flights                  0
Distance                       0
Points_Accumulated             0
Points_Redeemed                0
Dollar_Cost_Points_Redeemed    0
dtype: int64

## **Fase 2**: Visualización

In [200]:
# Imprimo las columnas numéricas 
columnas_numericas = df_customers.select_dtypes(include=['number']).columns

print("Las columnas numéricas son:", columnas_numericas)

Las columnas numéricas son: Index(['Loyalty_Number', 'Salary', 'CLV', 'Enrollment_Year',
       'Enrollment_Month', 'Year', 'Month', 'Flights_Booked',
       'Flights_with_Companions', 'Total_Flights', 'Distance',
       'Points_Accumulated', 'Points_Redeemed', 'Dollar_Cost_Points_Redeemed'],
      dtype='object')


In [201]:
# Imprimo las columnas categóricas 
columnas_categoricas = df_customers.select_dtypes(include=['object', 'category']).columns
print("Las columnas categóricas son:", columnas_categoricas)

Las columnas categóricas son: Index(['Province', 'City', 'Postal_Code', 'Gender', 'Education',
       'Marital_Status', 'Loyalty_Card', 'Enrollment_Type'],
      dtype='object')


1. ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?